# Módulo 5: IA Generativa con Amazon Bedrock

**Duración**: 30 minutos  
**Objetivo**: Dominar la IA generativa para crear contenido científico y código usando Amazon Bedrock

## ¿Qué aprenderás?
- **Parte A - Generación de Texto**: Crear reportes científicos, resúmenes e informes técnicos
- **Parte B - Generación de Código**: Desarrollar scripts de análisis de datos y funciones científicas
- Usar diferentes modelos de IA generativa (Claude, Titan)
- Aplicar prompting efectivo para casos científicos
- Configurar parámetros avanzados (temperatura, top-p)
- Comparar resultados entre modelos

## Configuración Inicial

In [ ]:
# Instalar dependencias
import sys
!{sys.executable} -m pip install boto3 ipywidgets matplotlib pandas numpy --quiet

print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import boto3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, Markdown, Code

# Configurar estilo
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Librerías importadas")

In [ ]:
# Configurar cliente de Amazon Bedrock
try:
    bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')
    print("✅ Cliente Bedrock configurado correctamente")
    print("✅ Usando credenciales IAM automáticas de SageMaker")
except Exception as e:
    print("❌ Error: Verificar permisos IAM para Bedrock")
    print(f"Error: {e}")

## Modelos Disponibles en Bedrock

Configuraremos diferentes modelos para comparar sus capacidades.

In [ ]:
# Modelos disponibles en Bedrock
MODELOS_BEDROCK = {
    "claude_haiku": {
        "id": "anthropic.claude-3-haiku-20240307-v1:0",
        "nombre": "Claude 3 Haiku",
        "fortaleza": "Rápido y eficiente",
        "uso": "Tareas simples, código básico"
    },
    "claude_sonnet": {
        "id": "anthropic.claude-3-sonnet-20240229-v1:0",
        "nombre": "Claude 3 Sonnet",
        "fortaleza": "Balance velocidad/calidad",
        "uso": "Análisis complejo, reportes"
    },
    "titan_text": {
        "id": "amazon.titan-text-express-v1",
        "nombre": "Amazon Titan Text",
        "fortaleza": "Optimizado para AWS",
        "uso": "Resúmenes, clasificación"
    }
}

print("Modelos configurados para el laboratorio:")
for key, modelo in MODELOS_BEDROCK.items():
    print(f"  • {modelo['nombre']}: {modelo['fortaleza']}")

# PARTE A: GENERACIÓN DE CONTENIDO CIENTÍFICO

## Casos de Uso para Generación de Texto

In [ ]:
# Casos de uso para generación de texto científico
CASOS_TEXTO = {
    "reporte_sismico": {
        "prompt": "Genera un reporte técnico sobre un sismo de magnitud 6.2 ocurrido en la región de Ica, Perú. Incluye: ubicación exacta, profundidad, intensidad, daños preliminares y recomendaciones.",
        "tipo": "reporte_técnico",
        "longitud": "500-700 palabras"
    },
    "resumen_investigacion": {
        "prompt": "Crea un resumen ejecutivo sobre los avances en predicción sísmica usando machine learning. Enfócate en: metodologías actuales, precisión alcanzada, limitaciones y futuras direcciones de investigación.",
        "tipo": "resumen_ejecutivo",
        "longitud": "300-400 palabras"
    },
    "protocolo_emergencia": {
        "prompt": "Desarrolla un protocolo de emergencia para instituciones científicas durante eventos sísmicos. Incluye: procedimientos de evacuación, protección de equipos, comunicaciones y continuidad operativa.",
        "tipo": "protocolo_operativo",
        "longitud": "400-500 palabras"
    }
}

print("Casos de uso para generación de texto científico:")
for nombre, caso in CASOS_TEXTO.items():
    print(f"  • {nombre.replace('_', ' ').title()}: {caso['tipo']}")

## Motor de Generación de Texto

In [ ]:
def generar_contenido_cientifico(prompt, modelo_key="claude_haiku", temperatura=0.7, max_tokens=1000):
    """
    Genera contenido científico usando diferentes modelos de Bedrock
    """
    modelo = MODELOS_BEDROCK[modelo_key]
    print(f"\n🤖 Generando con {modelo['nombre']}...")
    
    try:
        if "claude" in modelo_key:
            # Formato para modelos Claude
            body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "temperature": temperatura,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            })
        else:
            # Formato para Titan
            body = json.dumps({
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": max_tokens,
                    "temperature": temperatura,
                    "topP": 0.9
                }
            })
        
        response = bedrock.invoke_model(
            modelId=modelo["id"],
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        
        if "claude" in modelo_key:
            contenido = response_body['content'][0]['text']
        else:
            contenido = response_body['results'][0]['outputText']
        
        print(f"✓ Contenido generado: {len(contenido)} caracteres")
        
        return {
            'contenido': contenido,
            'modelo': modelo['nombre'],
            'parametros': {'temperatura': temperatura, 'max_tokens': max_tokens},
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except Exception as e:
        print(f"✗ Error: {e}")
        return None

print("✅ Motor de generación de texto configurado")

## Ejercicio A1: Generar Reporte Sísmico

In [ ]:
# Generar reporte sísmico
caso_reporte = CASOS_TEXTO['reporte_sismico']
reporte_resultado = generar_contenido_cientifico(
    caso_reporte['prompt'],
    modelo_key="claude_sonnet",
    temperatura=0.3,  # Más conservador para reportes técnicos
    max_tokens=800
)

if reporte_resultado:
    print("\n📄 REPORTE SÍSMICO GENERADO:")
    print("=" * 50)
    display(Markdown(reporte_resultado['contenido']))
    
    print(f"\n📊 Detalles de generación:")
    print(f"• Modelo: {reporte_resultado['modelo']}")
    print(f"• Temperatura: {reporte_resultado['parametros']['temperatura']}")
    print(f"• Caracteres: {len(reporte_resultado['contenido'])}")
else:
    print("No se pudo generar el reporte")

## Ejercicio A2: Generar Resumen de Investigación

In [ ]:
# Generar resumen de investigación
caso_resumen = CASOS_TEXTO['resumen_investigacion']
resumen_resultado = generar_contenido_cientifico(
    caso_resumen['prompt'],
    modelo_key="claude_haiku",
    temperatura=0.5,
    max_tokens=600
)

if resumen_resultado:
    print("\n📋 RESUMEN DE INVESTIGACIÓN GENERADO:")
    print("=" * 50)
    display(Markdown(resumen_resultado['contenido']))
    
    print(f"\n📊 Detalles de generación:")
    print(f"• Modelo: {resumen_resultado['modelo']}")
    print(f"• Temperatura: {resumen_resultado['parametros']['temperatura']}")
    print(f"• Caracteres: {len(resumen_resultado['contenido'])}")
else:
    print("No se pudo generar el resumen")

# PARTE B: GENERACIÓN DE CÓDIGO

## Casos de Uso para Generación de Código

In [ ]:
# Casos de uso para generación de código científico
CASOS_CODIGO = {
    "analisis_sismico": {
        "descripcion": "Crear una función que analice datos sísmicos y calcule la magnitud promedio, desviación estándar y identifique eventos anómalos (magnitud > 5.0)",
        "datos_ejemplo": "Lista de magnitudes sísmicas: [3.2, 4.1, 2.8, 5.5, 3.7, 4.2, 6.1, 3.9, 4.5, 3.1]",
        "tipo": "análisis_estadístico"
    },
    "visualizacion_geologica": {
        "descripcion": "Generar código para crear un gráfico de dispersión que muestre la relación entre profundidad y magnitud de sismos, con colores según la intensidad",
        "datos_ejemplo": "DataFrame con columnas: profundidad, magnitud, intensidad",
        "tipo": "visualización"
    },
    "procesamiento_señales": {
        "descripcion": "Crear una función que aplique un filtro pasa-bajas a señales sísmicas para eliminar ruido de alta frecuencia usando scipy",
        "datos_ejemplo": "Array numpy con datos de aceleración en el tiempo",
        "tipo": "procesamiento_señales"
    }
}

print("Casos de uso para generación de código científico:")
for nombre, info in CASOS_CODIGO.items():
    print(f"  • {nombre.replace('_', ' ').title()}: {info['tipo']}")

## Motor de Generación de Código

In [ ]:
def generar_codigo_cientifico(descripcion_tarea, tipo_codigo, datos_ejemplo=""):
    """
    Genera código Python para tareas científicas usando IA
    """
    print(f"\n🤖 Generando código para: {descripcion_tarea[:50]}...")
    print(f"Tipo: {tipo_codigo}")
    
    # Crear prompt especializado para código científico
    prompt = f"""Eres un experto en programación científica con Python. 
    
Tarea: {descripcion_tarea}
Tipo de código: {tipo_codigo}
Datos de ejemplo: {datos_ejemplo}

Genera código Python que:
1. Sea funcional y ejecutable
2. Incluya comentarios explicativos
3. Use librerías científicas apropiadas (numpy, pandas, matplotlib, scipy)
4. Maneje errores básicos
5. Incluya un ejemplo de uso con los datos proporcionados

Responde SOLO con el código Python, sin explicaciones adicionales:"""
    
    try:
        # Usar Claude 3 Haiku para generación de código
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "temperature": 0.2,  # Más determinístico para código
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        })
        
        response = bedrock.invoke_model(
            modelId="anthropic.claude-3-haiku-20240307-v1:0",
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        codigo_generado = response_body['content'][0]['text']
        
        print(f"✓ Código generado exitosamente")
        print(f"✓ Líneas de código: {len(codigo_generado.split())}")
        
        return {
            'codigo': codigo_generado,
            'descripcion': descripcion_tarea,
            'tipo': tipo_codigo,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except Exception as e:
        print(f"✗ Error en generación: {e}")
        return None

def ejecutar_codigo_generado(codigo_resultado):
    """
    Ejecuta el código generado de manera segura
    """
    if not codigo_resultado:
        print("❌ No hay código para ejecutar")
        return False
    
    print(f"\n🚀 Ejecutando código generado...")
    
    try:
        # Ejecutar el código en el contexto actual
        exec(codigo_resultado['codigo'], globals())
        print(f"✅ Código ejecutado exitosamente")
        return True
    except Exception as e:
        print(f"❌ Error en ejecución: {e}")
        return False

print("✅ Motor de generación de código configurado")

## Ejercicio B1: Generar Análisis Estadístico de Datos Sísmicos

In [ ]:
# Generar código para análisis sísmico
caso_sismico = CASOS_CODIGO['analisis_sismico']
codigo_sismico = generar_codigo_cientifico(
    caso_sismico['descripcion'],
    caso_sismico['tipo'],
    caso_sismico['datos_ejemplo']
)

if codigo_sismico:
    print("\n📝 CÓDIGO GENERADO:")
    print("=" * 50)
    display(Code(codigo_sismico['codigo'], language='python'))
    
    # Ejecutar el código generado
    ejecutar_codigo_generado(codigo_sismico)
else:
    print("No se pudo generar el código")

## Ejercicio B2: Generar Visualización Geológica

In [ ]:
# Generar código para visualización
caso_viz = CASOS_CODIGO['visualizacion_geologica']
codigo_viz = generar_codigo_cientifico(
    caso_viz['descripcion'],
    caso_viz['tipo'],
    caso_viz['datos_ejemplo']
)

if codigo_viz:
    print("\n📊 CÓDIGO DE VISUALIZACIÓN GENERADO:")
    print("=" * 50)
    display(Code(codigo_viz['codigo'], language='python'))
    
    # Ejecutar el código generado
    ejecutar_codigo_generado(codigo_viz)
else:
    print("No se pudo generar el código de visualización")

## Comparación de Modelos

In [ ]:
# Comparar diferentes modelos para la misma tarea
prompt_comparacion = "Explica en 100 palabras qué es la sismología y su importancia para la sociedad."

print("🔄 COMPARANDO MODELOS DE BEDROCK")
print("=" * 50)

modelos_comparar = ['claude_haiku', 'claude_sonnet']
resultados_comparacion = {}

for modelo_key in modelos_comparar:
    print(f"\n🤖 Generando con {MODELOS_BEDROCK[modelo_key]['nombre']}...")
    resultado = generar_contenido_cientifico(
        prompt_comparacion,
        modelo_key=modelo_key,
        temperatura=0.5,
        max_tokens=200
    )
    
    if resultado:
        resultados_comparacion[modelo_key] = resultado
        print(f"\n**{resultado['modelo']}:**")
        display(Markdown(resultado['contenido']))
        print(f"Caracteres: {len(resultado['contenido'])}")

print("\n✅ Comparación completada")

## Resumen del Laboratorio Bedrock

In [ ]:
# Resumen completo del laboratorio
print("📈 RESUMEN COMPLETO - AMAZON BEDROCK")
print("=" * 60)

# Parte A - Contenido generado
contenidos_generados = []
if 'reporte_resultado' in globals() and reporte_resultado:
    contenidos_generados.append(('Reporte Sísmico', reporte_resultado))
if 'resumen_resultado' in globals() and resumen_resultado:
    contenidos_generados.append(('Resumen Investigación', resumen_resultado))

print(f"\n📄 PARTE A - GENERACIÓN DE TEXTO:")
total_caracteres_texto = 0
for nombre, resultado in contenidos_generados:
    caracteres = len(resultado['contenido'])
    total_caracteres_texto += caracteres
    print(f"• {nombre}: {caracteres} caracteres ({resultado['modelo']})")

# Parte B - Código generado
codigos_generados = []
if 'codigo_sismico' in globals() and codigo_sismico:
    codigos_generados.append(('Análisis Sísmico', codigo_sismico))
if 'codigo_viz' in globals() and codigo_viz:
    codigos_generados.append(('Visualización Geológica', codigo_viz))

print(f"\n💻 PARTE B - GENERACIÓN DE CÓDIGO:")
total_lineas_codigo = 0
for nombre, codigo in codigos_generados:
    lineas = len(codigo['codigo'].split('\n'))
    total_lineas_codigo += lineas
    print(f"• {nombre}: {lineas} líneas de código ({codigo['tipo']})")

print(f"\n🎯 TOTALES:")
print(f"• Contenido científico generado: {len(contenidos_generados)} documentos")
print(f"• Código científico generado: {len(codigos_generados)} scripts")
print(f"• Total caracteres de texto: {total_caracteres_texto}")
print(f"• Total líneas de código: {total_lineas_codigo}")
print(f"• Modelos utilizados: Claude 3 Haiku, Claude 3 Sonnet")

print(f"\n🔧 CAPACIDADES DEMOSTRADAS:")
capacidades = [
    'Generación de reportes técnicos',
    'Creación de resúmenes ejecutivos',
    'Desarrollo de código de análisis estadístico',
    'Generación de scripts de visualización',
    'Comparación entre modelos de IA',
    'Configuración de parámetros avanzados'
]

for capacidad in capacidades:
    print(f"• {capacidad}")

print("\n✓ Laboratorio Amazon Bedrock completado exitosamente")

## Casos de Uso en Investigación Científica

### Amazon Bedrock para Ciencias:
- **Generación de reportes**: Crear informes técnicos y documentación científica
- **Resúmenes de literatura**: Sintetizar papers y estudios de investigación
- **Traducción técnica**: Adaptar contenido científico entre idiomas
- **Generación de código**: Crear scripts de análisis y procesamiento de datos
- **Documentación automática**: Generar comentarios y documentación de código
- **Análisis de datos**: Desarrollar algoritmos de procesamiento científico
- **Protocolos operativos**: Crear procedimientos y guías técnicas
- **Comunicación científica**: Adaptar contenido técnico para diferentes audiencias

## Validación del Módulo

In [ ]:
def validar_modulo_bedrock():
    verificaciones = {
        "Bedrock configurado": 'bedrock' in globals(),
        "Contenido científico generado": False,
        "Código científico generado": False,
        "Comparación de modelos realizada": False
    }
    
    # Verificar generación de contenido
    if 'reporte_resultado' in globals() and reporte_resultado:
        verificaciones["Contenido científico generado"] = len(reporte_resultado.get('contenido', '')) > 0
    
    # Verificar generación de código
    if 'codigo_sismico' in globals() and codigo_sismico:
        verificaciones["Código científico generado"] = len(codigo_sismico.get('codigo', '')) > 0
    
    # Verificar comparación de modelos
    if 'resultados_comparacion' in globals() and resultados_comparacion:
        verificaciones["Comparación de modelos realizada"] = len(resultados_comparacion) > 1
    
    print("VALIDACIÓN MÓDULO 5 - AMAZON BEDROCK")
    print("=" * 45)
    
    for verificacion, estado in verificaciones.items():
        icono = "✓" if estado else "✗"
        print(f"{icono} {verificacion}")
    
    completadas = sum(verificaciones.values())
    total = len(verificaciones)
    porcentaje = (completadas/total)*100
    
    print(f"\nProgreso: {completadas}/{total} ({porcentaje:.0f}%)")
    
    if completadas >= 3:
        print("\n🎉 ¡MÓDULO BEDROCK COMPLETADO!")
        print("Has dominado la IA generativa para contenido y código científico")
        print("🏁 ¡LABORATORIO COMPLETO! Has explorado todos los servicios de AWS AI/ML")
    else:
        print("\n⚠️ Completa más ejercicios de IA generativa")
    
    return completadas >= 3

modulo_completado = validar_modulo_bedrock()

---

## ¡Laboratorio Completado!

**Has explorado 5 servicios de AWS AI/ML:**
1. **Amazon Rekognition** - Análisis de imágenes satelitales
2. **Amazon Comprehend** - Procesamiento de texto científico
3. **Amazon Textract** - OCR inteligente de documentos
4. **Amazon Polly** - Síntesis de voz para alertas
5. **Amazon Bedrock** - IA generativa para contenido y código científico

### 🎯 Logros Alcanzados:
- Procesamiento de imágenes geológicas
- Análisis de documentos científicos
- Digitalización de formularios y tablas
- Creación de alertas de audio
- Generación de reportes técnicos
- Desarrollo de código científico con IA

---

*Módulo 5 de 5 completado*